In [1]:
import pandas as pd
from sqlalchemy import create_engine, Integer, VARCHAR
from util import consts
from collections import Counter
from seg import segmentation 


engine = create_engine(consts.MYSQL_CONNECTION_PMS)
connect = engine.connect()

In [107]:
from elasticsearch import Elasticsearch
es = Elasticsearch('http://localhost:9200')
result_dict = es.search(index="pms_story", body={
    "query":{
        "constant_score":{
                    "filter": {
                        "range": {
                              "story": {
                                    "gt": 1,
                                    "lt": 21
                                            }
                                                }
                                                  }
                                                    }
                                                     }
                                                        }, size=1000)

from pandasticsearch import Select
pandas_df = Select.from_dict(result_dict).to_pandas()[['story', 'title', 'spec']]
pandas_df

,story,title,spec
0,2,特定情况下的外框最后排序问题,"类似于\n打标图层\n:\n开放一个\n最后切割图层\n, \n以弥补和应对\n新排序原则下..."
1,16,外部人机界面状态显示,作为产品经理，我希望可以在使用外部人机界面，例如IO、数控面板时，能够在外部人机界面上显示当...
2,5,CypOne动态速度算法,\n1. 在不同速度下算法的切换。\n\r\n\n 1）速度 < 10mmm/s ...
3,15,外部人机界面控制,\n作为产品经理，我希望CypCut可以外接各类人机界面，包括：\n\r\n\n\r\n\n...
4,18,武汉天琪关于圆管切割的2个需求。,\n需求1：\n\r\n\n圆管的管径测量不准，有的地方是49mm，有的地方是51mm。需要...
5,14,1205卡的扫描切割功能,1205卡搭配cypcut使用时，开启扫描切割功能，方便部分客户在某些场合的加工需求。
6,20,人机界面的独立性,作为用户，我希望各种人机界面之间的操作能够互不干扰。在我使用手持盒进行点动时，使用手持盒上的...
7,7,3点自动寻边,\n优化目前的自动寻边功能，支持可以3点寻边。可以不仅找出板子倾斜的角度，还可以精确的找到板...
8,6,程序初始化时在任务栏显示,在软件运行的整个过程中，Windows任务栏能够始终显示程序。不应该出现程序弹出了一个对话框...
9,13,以截止时间来进行加密,作为设备制造商，我希望可以在对BCS100进行加密时使用截止日期而不是天数，并且每次注册之后...


In [106]:
result_dict['hits']['hits']

[]

In [35]:
sql = 'select story, title, spec from zt_storyspec where story > {0}'.format(1000)
story_df = pd.read_sql(sql, engine)
from nohtml import strip_tags
story_df['spec'] = story_df['spec'].apply(lambda x:strip_tags(x))
story_df = story_df.loc[story_df['story'].drop_duplicates().index,:]

story_df = story_df.set_index(story_df['story'].values)

story_df

,story,title,spec
1001,1001,Hyppanel触屏体验提升,作为一名产品经理，\n 在使用Hyppanel操作Hypcut时，时常会出现手指...
1002,1002,图形设置飞切以后可以编辑修改,在CypDraw的图纸处理和零件编辑里，被设置为飞切的图形都没有任何方式打散来做修改。\n希...
1003,1003,圆弧长条的排样结果让人吐血,打开附件nsd后，自动排样\n圆弧长条零件的排样效率太低下了，简直令人发指\n
1004,1004,Lite版本可以修改倒角尺寸,作为力星的康泽，我们客户在绘制图形的时候经常画错倒角。\n使用3000系统时可以修改倒角进行...
1005,1005,威视、映美精、basler、海康四种工业相机的对比测试,相机作为外部资源，存在着不可控的因素。所以，我希望能够将我们现在支持的四种相机：威视、映美精...
...,...,...,...
4647,4647,设计喷码装机界面，将原调试界面中属于第一次装机才设置的参数提取出来。,【内部迭代需求】\n 基于功能点文档优化界面显示。功能点文档参考：http:...
4648,4648,优化喷码调试界面。,【内部迭代需求】\n 基于功能点文档优化界面显示。功能点文档参考：http:...
4649,4649,增加断点续喷的功能。,【内部迭代需求】\n 在喷码的过程中可能会遇到客户人为暂停或者停止，也可...
4650,4650,提供只喷码，只加工，喷码并加工三种加工方式。,【内部迭代需求】\n 在切割系统中喷码是一个非必须的操作，即使文件里面包含...


In [8]:
dict(story_df.loc[1,:])

{'story': 1,
 'title': '加密狗选型',
 'spec': '选择一个合适的加密狗，带有硬件时钟，稳定性好，供应商信誉好，供货稳定，价格偏移。'}

In [32]:
words = Counter()
words['n'] = len(story_df)
story_df['seg'] = ''
for i in range(1, len(story_df) + 1):
    seg = segmentation((story_df['title'][i] + story_df['spec'][i]).split())
    word_count = Counter(seg)
    print(i)
    print(word_count)
    wordn = sum(word_count.values())
    for word in word_count:
        word_count[word] = word_count[word]/wordn
    print(word_count)
        
    words += Counter(word_count.keys())
    story_df['seg'][i] = word_count
    

1
Counter({'加密狗': 2, '选型': 1, '硬件': 1, '时钟': 1, '稳定性': 1, '信誉': 1, '供货': 1, '价格': 1})
Counter({'加密狗': 0.2222222222222222, '选型': 0.1111111111111111, '硬件': 0.1111111111111111, '时钟': 0.1111111111111111, '稳定性': 0.1111111111111111, '信誉': 0.1111111111111111, '供货': 0.1111111111111111, '价格': 0.1111111111111111})
2
Counter({'排序': 3, '外框': 2, '原则': 2, '标图': 1, '图层': 1, '图形': 1})
Counter({'排序': 0.3, '外框': 0.2, '原则': 0.2, '标图': 0.1, '图层': 0.1, '图形': 0.1})
3
Counter({'板材': 3, '规则': 2, '选项': 1, '水平度': 1, '曲面': 1, '模式': 1, '带膜': 1})
Counter({'板材': 0.3, '规则': 0.2, '选项': 0.1, '水平度': 0.1, '曲面': 0.1, '模式': 0.1, '带膜': 0.1})
4
Counter({'工作台': 5, '按钮': 3, '气缸': 3, '手工': 2, '动作': 2, '信号': 2, '控制卡': 2, '由天琪': 1, '迪能': 1, '钢板': 1, '硬件': 1, '条件': 1, '行程开关': 1})
Counter({'工作台': 0.2, '按钮': 0.12, '气缸': 0.12, '手工': 0.08, '动作': 0.08, '信号': 0.08, '控制卡': 0.08, '由天琪': 0.04, '迪能': 0.04, '钢板': 0.04, '硬件': 0.04, '条件': 0.04, '行程开关': 0.04})
5
Counter({'加速度': 7, 'FBFW3': 5, 'mmm': 4, '滤波器': 2, '控制精度': 2, 'CypOne': 1, '动态': 1

d:\Users\fscut\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app



Counter({'人机界面': 3, '经理': 1, 'CypCut': 1, '无线': 1, '面板': 1, '场合': 1, '体验': 1})
Counter({'人机界面': 0.3333333333333333, '经理': 0.1111111111111111, 'CypCut': 0.1111111111111111, '无线': 0.1111111111111111, '面板': 0.1111111111111111, '场合': 0.1111111111111111, '体验': 0.1111111111111111})
16
Counter({'人机界面': 3, '状态': 2, '经理': 1, '面板': 1})
Counter({'人机界面': 0.42857142857142855, '状态': 0.2857142857142857, '经理': 0.14285714285714285, '面板': 0.14285714285714285})
17
Counter({'动作': 1})
Counter({'动作': 1.0})
18
Counter({'圆管': 4, '天琪': 2, '地方': 2, '间距': 2, '旋转轴': 2, '武汉': 1, '管径': 1, '角度': 1, '编程': 1, '方管': 1, '全局': 1, '减速比': 1})
Counter({'圆管': 0.21052631578947367, '天琪': 0.10526315789473684, '地方': 0.10526315789473684, '间距': 0.10526315789473684, '旋转轴': 0.10526315789473684, '武汉': 0.05263157894736842, '管径': 0.05263157894736842, '角度': 0.05263157894736842, '编程': 0.05263157894736842, '方管': 0.05263157894736842, '全局': 0.05263157894736842, '减速比': 0.05263157894736842})
19
Counter({'开发人员': 2, '体验': 2, '键盘': 1, '强制性': 1}

71
Counter({'双驱': 1, '专业': 1, '米左右': 1, '特作': 1, '主动轴': 1, '动轴': 1, '伺服电机': 1})
Counter({'双驱': 0.14285714285714285, '专业': 0.14285714285714285, '米左右': 0.14285714285714285, '特作': 0.14285714285714285, '主动轴': 0.14285714285714285, '动轴': 0.14285714285714285, '伺服电机': 0.14285714285714285})
72
Counter({'单位': 3, 'min': 2, '全局': 1, '参数设置': 1, '习惯': 1})
Counter({'单位': 0.375, 'min': 0.25, '全局': 0.125, '参数设置': 0.125, '习惯': 0.125})
73
Counter()
Counter()
74
Counter({'触点': 2, '按钮': 2, '类型': 1, '选项': 1, '武汉': 1, '高能': 1})
Counter({'触点': 0.25, '按钮': 0.25, '类型': 0.125, '选项': 0.125, '武汉': 0.125, '高能': 0.125})
75
Counter({'BMC1204': 2, '脉冲': 2, '平率': 1, '频率': 1, 'KHz': 1, 'MHz': 1})
Counter({'BMC1204': 0.25, '脉冲': 0.25, '平率': 0.125, '频率': 0.125, 'KHz': 0.125, 'MHz': 0.125})
76
Counter({'单位': 2, '英寸': 2, '市场': 1, 'min': 1, '比例': 1})
Counter({'单位': 0.2857142857142857, '英寸': 0.2857142857142857, '市场': 0.14285714285714285, 'min': 0.14285714285714285, '比例': 0.14285714285714285})
77
Counter({'CYPCUT': 4, 'Coreldr

110
Counter({'区域': 2, '管材': 2, '状态': 2, '动作': 2, 'PLC': 1, '条件': 1, '平面': 1, '方向': 1, '坐标值': 1, '方管': 1, '支架': 1, '设定值': 1, '手动': 1, '金运': 1})
Counter({'区域': 0.1111111111111111, '管材': 0.1111111111111111, '状态': 0.1111111111111111, '动作': 0.1111111111111111, 'PLC': 0.05555555555555555, '条件': 0.05555555555555555, '平面': 0.05555555555555555, '方向': 0.05555555555555555, '坐标值': 0.05555555555555555, '方管': 0.05555555555555555, '支架': 0.05555555555555555, '设定值': 0.05555555555555555, '手动': 0.05555555555555555, '金运': 0.05555555555555555})
111
Counter({'图层': 1, '高度': 1, '方管': 1, '平面': 1, '区分': 1})
Counter({'图层': 0.2, '高度': 0.2, '方管': 0.2, '平面': 0.2, '区分': 0.2})
112
Counter({'加密狗': 3, '联系': 2, '制造商': 1, '老板': 1, '交款': 1, '伤感情': 1, '人情味': 1, '加密': 1, '人性化': 1, '陈大健': 1})
Counter({'加密狗': 0.23076923076923078, '联系': 0.15384615384615385, '制造商': 0.07692307692307693, '老板': 0.07692307692307693, '交款': 0.07692307692307693, '伤感情': 0.07692307692307693, '人情味': 0.07692307692307693, '加密': 0.07692307692307693, '人性化': 

...

807
Counter({'长孔': 10, '线性': 8, '寻中': 5, '管心': 4, '小孔': 4, '偏差': 2, '层面': 2, 'ABC': 2, '普通': 2, '庆源': 1, '地方': 1, '管材': 1, '技术': 1, '轮廓线': 1, '补点': 1, '交叉': 1, '时点': 1, '边界': 1, '模型': 1, '对象': 1, '盒内': 1, '长线': 1, '精准': 1, '距离': 1, '要点': 1, 'ZZX': 1, '文件': 1, '个长': 1, '范围': 1})
Counter({'长孔': 0.1694915254237288, '线性': 0.13559322033898305, '寻中': 0.0847457627118644, '管心': 0.06779661016949153, '小孔': 0.06779661016949153, '偏差': 0.03389830508474576, '层面': 0.03389830508474576, 'ABC': 0.03389830508474576, '普通': 0.03389830508474576, '庆源': 0.01694915254237288, '地方': 0.01694915254237288, '管材': 0.01694915254237288, '技术': 0.01694915254237288, '轮廓线': 0.01694915254237288, '补点': 0.01694915254237288, '交叉': 0.01694915254237288, '时点': 0.01694915254237288, '边界': 0.01694915254237288, '模型': 0.01694915254237288, '对象': 0.01694915254237288, '盒内': 0.01694915254237288, '长线': 0.01694915254237288, '精准': 0.01694915254237288, '距离': 0.01694915254237288, '要点': 0.01694915254237288, 'ZZX': 0.01694915254237288, '文件': 0.0

863
Counter({'支架': 13, '图形': 3, '原点': 2, '偏差': 2, '管材': 1, '悬空': 1, '脉冲': 1, '当量': 1, '限位': 1, '页面': 1, '发送给': 1, '管心': 1, '一键': 1, '手动': 1, '条件': 1})
Counter({'支架': 0.41935483870967744, '图形': 0.0967741935483871, '原点': 0.06451612903225806, '偏差': 0.06451612903225806, '管材': 0.03225806451612903, '悬空': 0.03225806451612903, '脉冲': 0.03225806451612903, '当量': 0.03225806451612903, '限位': 0.03225806451612903, '页面': 0.03225806451612903, '发送给': 0.03225806451612903, '管心': 0.03225806451612903, '一键': 0.03225806451612903, '手动': 0.03225806451612903, '条件': 0.03225806451612903})
864
Counter({'终端用户': 1})
Counter({'终端用户': 1.0})
865
Counter({'定位': 2, '运行日志': 1, '页面': 1, '使用者': 1, '目光': 1})
Counter({'定位': 0.3333333333333333, '运行日志': 0.16666666666666666, '页面': 0.16666666666666666, '使用者': 0.16666666666666666, '目光': 0.16666666666666666})
866
Counter({'里程': 3, '润滑油': 2, '自润滑': 1, '因素': 1, '直接判断': 1, '延时': 1})
Counter({'里程': 0.3333333333333333, '润滑油': 0.2222222222222222, '自润滑': 0.1111111111111111, '因素': 0.11111111

Counter({'BLDC': 0.2, 'Khz': 0.2, 'PWM': 0.2, '增益': 0.2, '加速度': 0.2})
906
Counter({'信号': 3, '方向': 2, 'BLDC': 1, 'PWM': 1})
Counter({'信号': 0.42857142857142855, '方向': 0.2857142857142857, 'BLDC': 0.14285714285714285, 'PWM': 0.14285714285714285})
907
Counter({'licence': 2, 'RTOS': 1, '经理': 1, '硬件': 1, '加密狗': 1, '稳定性': 1})
Counter({'licence': 0.2857142857142857, 'RTOS': 0.14285714285714285, '经理': 0.14285714285714285, '硬件': 0.14285714285714285, '加密狗': 0.14285714285714285, '稳定性': 0.14285714285714285})
908
Counter({'逻辑': 1})
Counter({'逻辑': 1.0})
909
Counter({'十字': 6, 'mark': 6, '类型': 4, '市场部': 1, 'CAD': 1, '直线': 1})
Counter({'十字': 0.3157894736842105, 'mark': 0.3157894736842105, '类型': 0.21052631578947367, '市场部': 0.05263157894736842, 'CAD': 0.05263157894736842, '直线': 0.05263157894736842})
910
Counter({'光斑': 4, '校准': 2, '平台': 2, '何鑫峰': 1, '样机': 1, '短时间': 1, '数值': 1, '范围': 1})
Counter({'光斑': 0.3076923076923077, '校准': 0.15384615384615385, '平台': 0.15384615384615385, '何鑫峰': 0.07692307692307693, '样机':

946
Counter({'引线': 3, '起点': 2, '方位': 2, '江门': 1, '刀路': 1, '矩形': 1, '产生': 1, '智格': 1, 'SigmaNest': 1, '选项': 1, 'Alma': 1})
Counter({'引线': 0.2, '起点': 0.13333333333333333, '方位': 0.13333333333333333, '江门': 0.06666666666666667, '刀路': 0.06666666666666667, '矩形': 0.06666666666666667, '产生': 0.06666666666666667, '智格': 0.06666666666666667, 'SigmaNest': 0.06666666666666667, '选项': 0.06666666666666667, 'Alma': 0.06666666666666667})
947
Counter({'排序': 4, '手动': 2, '加进来': 1, '区分': 1, '拉线': 1, '方向键': 1, 'CypDraw': 1})
Counter({'排序': 0.36363636363636365, '手动': 0.18181818181818182, '加进来': 0.09090909090909091, '区分': 0.09090909090909091, '拉线': 0.09090909090909091, '方向键': 0.09090909090909091, 'CypDraw': 0.09090909090909091})
948
Counter({'梯形': 1, '附件': 1, 'DXF': 1, 'String': 1, 'MultiA': 1, '板材': 1, '间距': 1, 'CypDraw': 1, 'SigmaNest': 1})
Counter({'梯形': 0.1111111111111111, '附件': 0.1111111111111111, 'DXF': 0.1111111111111111, 'String': 0.1111111111111111, 'MultiA': 0.1111111111111111, '板材': 0.1111111111111111

972
Counter({'支架': 39, '总阀': 14, '手动': 7, '状态': 7, '关联': 3, '逻辑': 3, '圆管': 3, 'ZZX': 3, 'tubepro': 2, '平台': 2, '页面': 2, '方管': 2, '信号': 1, '动作': 1, '意义': 1, '选项': 1, '开机': 1, '宏山': 1, '气动': 1, '对方': 1, '文件': 1})
Counter({'支架': 0.40625, '总阀': 0.14583333333333334, '手动': 0.07291666666666667, '状态': 0.07291666666666667, '关联': 0.03125, '逻辑': 0.03125, '圆管': 0.03125, 'ZZX': 0.03125, 'tubepro': 0.020833333333333332, '平台': 0.020833333333333332, '页面': 0.020833333333333332, '方管': 0.020833333333333332, '信号': 0.010416666666666666, '动作': 0.010416666666666666, '意义': 0.010416666666666666, '选项': 0.010416666666666666, '开机': 0.010416666666666666, '宏山': 0.010416666666666666, '气动': 0.010416666666666666, '对方': 0.010416666666666666, '文件': 0.010416666666666666})
973
Counter()
Counter()
974
Counter({'WIN10': 4, '市场': 1, 'cypcut': 1})
Counter({'WIN10': 0.6666666666666666, '市场': 0.16666666666666666, 'cypcut': 0.16666666666666666})
975
Counter({'焦点': 3, '倍数': 2, '范围': 2, '智能': 1, '硬件': 1, '何鑫锋': 1, '光学': 1, '负责人': 

In [33]:
story_df.to_json('data/pms_story.json')

In [117]:
story_df = pd.read_json('data/pms_story.json')
story_df

,story,title,spec,seg
2,2,特定情况下的外框最后排序问题,"类似于\n打标图层\n:\n开放一个\n最后切割图层\n, \n以弥补和应对\n新排序原则下...","{'外框': 0.2, '排序': 0.30000000000000004, '标图': 0..."
30,30,小钥匙中可以初始化加密狗,作为设备制造商，我们的加密通常是由商务部分的人员来管理的，他们通常并没有安装软件，也就不能得...,"{'钥匙': 0.2, '加密狗': 0.2, '制造商': 0.1, '加密': 0.1,..."
16,16,外部人机界面状态显示,作为产品经理，我希望可以在使用外部人机界面，例如IO、数控面板时，能够在外部人机界面上显示当...,"{'人机界面': 0.4285714286, '状态': 0.2857142857, '经理..."
5,5,CypOne动态速度算法,\n1. 在不同速度下算法的切换。\n\r\n\n 1）速度 < 10mmm/s ...,"{'CypOne': 0.029411764700000002, '动态': 0.02941..."
184,184,方管独立于图形的R角处理,\n作为客户，在调试切割效果时经常遇到方管在倒角处工艺很难调试的问题，现在方管的旋转是依据R...,"{'方管': 0.2727272727, '图形': 0.1818181818, '倒角':..."
...,...,...,...,...
4448,4448,增加示教余料的功能,\n作为张卫杰，去戴南出差时，观察到一个现象\n\n\n\n\n\n\n\n\n现象：终端客...,"{'余料': 0.3157894737, '张卫杰': 0.0526315789, '戴南'..."
4449,4449,增加走包络线的功能,\n作为张卫杰，去戴南出差时，观察到一个现象\n\n\n\n\n\n\n\n\n现象：终端用...,"{'包络线': 0.0689655172, '张卫杰': 0.0344827586, '戴南..."
4450,4450,走边框时自动检测是否在板内,\n作为张卫杰，去戴南出差时，观察到一个现象\n\n\n\n\n\n\n\n\n\n现象：他...,"{'边框': 0.2105263158, '自动检测': 0.0526315789, '张卫..."
4451,4451,希望打开文件时，能删除零件内部的所有不加工的文字,\n嘉泰终端用户（戴南）反馈，我的出图流程依次是：\n\n\n\n\n\n\n1.客户来图加...,"{'文件': 0.0740740741, '文字': 0.2962962963, '嘉泰':..."


In [115]:
story_df = pd.read_json('data/pms_task.json')
story_df

,name,id,desc,seg
4,CypCut性能优化——打开文件优化,4,尽可能的提高CypCut打开文件的速度，包括文件读取、自动优化、曲线平滑、排序直到完全显示的过程。,"{'CypCut': 0.25, '性能': 0.125, '文件': 0.375, '曲线..."
101,方管截面的逆向工程,101,\n在读取IGS文件的过程中，对方管的截面进行逆向工程，识别出常见的管类型：\n\r\n\n...,"{'方管': 0.1538461538, '逆向': 0.1538461538, '工程':..."
214,增加右键设置MARK点,214,,"{'右键': 0.5, 'MARK': 0.5}"
107,使用逆向工程的截面进行法向量的计算,107,\n检查程序中法向量计算的部分，尽量使用逆向工程的界面进行法向量计算。\n\r\n\n\n\...,"{'逆向': 0.0909090909, '工程': 0.0909090909, '向量':..."
216,视觉算法测试工具,216,,{'测试工具': 1.0}
...,...,...,...,...
15238,[8.11] 测试求解块分块带来的性能提升,15238,\n[任务目标]\n\n\n刚体提供两个（或三个）未知数写完了，实际测试一下求解块分块带来的...,"{'分块': 0.1818181818, '性能': 0.2727272727, '目标':..."
15239,[7.0.15.135]通用轴回原点时碰到限位信号就会报错（nc.bpl）,15239,,"{'原点': 0.25, '限位': 0.25, '信号': 0.25, 'bpl': 0.25}"
15240,[3.0.30.0]独立Z轴，第二个视觉定位分组，Z轴没有运动到视觉停靠点就开始视觉定位,15240,,"{'定位': 0.6666666667000001, '停靠点': 0.3333333333..."
15241,优化状态监控中显示列表更新功能,15241,\n[任务目标] 开始监控有会修改监控显示列表，用户体验不好。\n\n\n[思路/步骤] 如...,"{'状态': 0.1, '列表': 0.4, '目标': 0.1, '体验': 0.1, '..."


In [119]:
story_df['story'].value_counts()

2047    1
2616    1
2604    1
557     1
2608    1
       ..
3343    1
1294    1
3339    1
1290    1
2049    1
Name: story, Length: 4452, dtype: int64

In [76]:
up_df['id']

1        1
2        2
3        3
4        4
5        5
      ... 
96      96
97      97
98      98
99      99
100    100
Name: id, Length: 100, dtype: int64

In [29]:
sql = 'select story, title, spec from zt_storyspec where story > {0}'.format(1)
sql


'select story, title, spec from zt_storyspec where story > 1'

In [19]:
from datetime import datetime
from elasticsearch import Elasticsearch


es = Elasticsearch('127.0.0.1:9200')

In [23]:
dict(up_df.loc[i, :])

{'story': 1,
 'title_text': '加密狗选型',
 'spec_text': '选择一个合适的加密狗，带有硬件时钟，稳定性好，供应商信誉好，供货稳定，价格偏移。',
 'keywords_keyword': ['加密狗', '时钟', '信誉', '供货', '选型', '价格', '稳定性', '硬件']}

In [25]:
import math
for i in range(1, len(story_df) + 1):
    tf = Counter(story_df['seg'][i].copy())
    for word in tf:
        tf[word] = round(tf[word]*math.log(words['n']/(words[word] + 1)), 2)
    tf = tf.most_common(10)
    print(tf)
    up_df['keywords'][i] = [w[0] for w in tf]
    print(i)
    
    es.index(index="pms_story_test_1", id=story_df['story'][i], body= dict(up_df.loc[i, :]))
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

[('加密狗', 1.13), ('时钟', 0.82), ('信誉', 0.82), ('供货', 0.82), ('选型', 0.67), ('价格', 0.59), ('稳定性', 0.51), ('硬件', 0.49)]
1


d:\Users\fscut\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


[('原则', 1.16), ('外框', 1.06), ('排序', 1.03), ('标图', 0.56), ('图层', 0.35), ('图形', 0.22)]
2
[('规则', 0.87), ('板材', 0.83), ('水平度', 0.78), ('曲面', 0.78), ('带膜', 0.78), ('模式', 0.3), ('选项', 0.28)]
3
[('工作台', 0.77), ('气缸', 0.55), ('手工', 0.41), ('控制卡', 0.4), ('按钮', 0.35), ('由天琪', 0.31), ('行程开关', 0.28), ('信号', 0.25), ('钢板', 0.23), ('动作', 0.22)]
4
[('FBFW3', 1.11), ('加速度', 0.91), ('mmm', 0.89), ('滤波器', 0.4), ('控制精度', 0.34), ('MaxAcc', 0.22), ('WorkSpeed', 0.22), ('Acc', 0.22), ('FIRFreq', 0.22), ('ArcTolerance', 0.22)]
5
[('任务栏', 1.71), ('Windows', 1.11), ('程序', 0.9), ('对话框', 0.5), ('CypCut', 0.45), ('Cypcut', 0.34)]
6
[('板边', 1.76), ('板子', 1.06), ('角度', 0.9), ('距离', 0.85)]
7
[('雷达', 3.42), ('图形', 1.11)]
8
[('功率', 1.49), ('曲线', 1.02), ('方法', 0.45), ('半速', 0.43), ('单调', 0.39), ('成功率', 0.35), ('规律', 0.35)]
9
[('高度', 2.39), ('方管', 1.41)]
10
[('圆管', 1.0), ('文字', 0.73), ('灯具', 0.65), ('造型', 0.65), ('艺术', 0.61), ('图形', 0.56), ('平面', 0.34)]
11
[('画面', 1.34), ('开机', 1.18), ('示教盒', 0.97), ('认知度', 0.97), ('出厂'

[('图形', 1.11), ('大乘', 1.11), ('选择性', 0.5), ('刀线', 0.45), ('中心', 0.29), ('长度', 0.25), ('穿孔', 0.23)]
93
[('开始菜单', 1.57), ('名称', 0.55), ('OEM', 0.53), ('快捷方式', 0.52), ('驱动程序', 0.5), ('桌面', 0.49), ('文件夹', 0.38), ('图标', 0.35), ('平台', 0.2), ('工具', 0.2)]
94
[('快捷方式', 2.94), ('桌面上', 1.55), ('开始菜单', 1.47), ('程序', 0.72)]
95
[('警告', 1.82), ('信号线', 0.97), ('电阻', 0.97), ('设计方案', 0.88), ('驱动器', 0.53), ('按钮', 0.36)]
96
[('烧角', 0.7), ('厚板', 0.68), ('拐角', 0.49), ('天泓', 0.41), ('陈兆', 0.41), ('很漂亮', 0.41), ('正方形', 0.33), ('床子', 0.31), ('拐角处', 0.29), ('尖角', 0.29)]
97
[('加密', 0.98), ('全局', 0.83), ('图层', 0.55), ('终端用户', 0.4), ('天泓汪', 0.31), ('工艺员', 0.31), ('厂里', 0.29), ('黄工', 0.22), ('出厂', 0.2), ('备份', 0.19)]
98
[('时序', 1.71), ('光闸', 1.62), ('大功率', 0.72), ('IPG', 0.52), ('ProgramStart', 0.43), ('全部', 0.23), ('动作', 0.17)]
99
[('电流', 1.26), ('峰值', 1.17), ('PWM', 0.71), ('功率', 0.52), ('穿孔', 0.44), ('大程度', 0.35), ('高功率', 0.21), ('形式', 0.19)]
100
[('QCW', 1.18), ('频率', 0.87), ('触摸屏', 0.43), ('大族', 0.27), ('乘积', 

[('手点', 1.22), ('制图员', 1.18), ('拐角处', 0.91), ('一键', 0.67), ('角度', 0.6), ('图形', 0.37)]
174
[('眼镜', 2.44), ('细边', 1.11), ('大端', 0.55), ('顺序', 0.31), ('区分', 0.26), ('角度', 0.26), ('方向', 0.24), ('附件', 0.21), ('图形', 0.16)]
175
[('编码器', 2.38), ('指向性', 0.7), ('缺相', 0.7), ('细化', 0.62), ('区分', 0.33), ('市场', 0.28)]
176
[('刀线', 1.22), ('图形', 0.43), ('方向', 0.41), ('区分', 0.26), ('CAD', 0.18), ('手点', 0.18), ('排序', 0.17), ('阳切', 0.17), ('渠道', 0.16), ('shift', 0.16)]
177
[('PWM', 0.94), ('脉冲', 0.63), ('继电器', 0.5), ('信号', 0.47), ('QCW', 0.39), ('频率', 0.33), ('短时间', 0.33), ('模式', 0.23), ('常量', 0.2), ('高灵敏度', 0.2)]
178
[('PLC', 0.66), ('频率', 0.62), ('先生', 0.55), ('穿孔', 0.46), ('电流', 0.4), ('PWM', 0.37), ('峰值', 0.37), ('材料', 0.28), ('功率', 0.27), ('选项', 0.2)]
179
[('Help', 0.89), ('文件夹', 0.81), ('总工', 0.3), ('须知', 0.3), ('冷水机', 0.3), ('故障诊断', 0.28), ('区域', 0.28), ('文件目录', 0.27), ('清空', 0.26), ('按钮', 0.22)]
180
[('sysinfo', 3.88), ('对话框', 1.49), ('命令', 1.27)]
181
[('穿孔', 1.92), ('节约', 0.97), ('厚板', 0.87)]
18

...

[('卷料', 2.96), ('修正', 0.62), ('图形', 0.3), ('一键', 0.27), ('体验', 0.24), ('按钮', 0.19)]
4288
[('CypVision', 0.81), ('误差', 0.76), ('Ctrl', 0.68), ('快捷键', 0.53), ('使用者', 0.48), ('菜单', 0.48), ('起点', 0.38), ('范围', 0.36)]
4289
[('UltraCut', 0.78), ('CypVision', 0.45), ('占用率', 0.35), ('CPU', 0.33), ('定位', 0.32), ('品牌', 0.31), ('竞争力', 0.31), ('进程', 0.3), ('关键', 0.28), ('模板', 0.25)]
4290
[('图片', 1.48), ('BCVMaster', 0.6), ('画面', 0.59), ('图像', 0.56), ('文件', 0.28), ('lxd', 0.26), ('现状', 0.24), ('开发人员', 0.23), ('CypCut', 0.15)]
4291
[('余料', 1.62), ('chm', 0.73), ('菜单栏', 0.64), ('BCVMaster', 0.54), ('现状', 0.44), ('形式', 0.42), ('选项', 0.28)]
4292
[('区域', 1.15), ('背景', 0.69), ('魔术', 0.67), ('画布', 0.67), ('图片', 0.4), ('bug16455', 0.35), ('现状', 0.2), ('轮廓', 0.19), ('余料', 0.18), ('标定', 0.16)]
4293
[('画布', 1.4), ('背景', 0.72), ('标定', 0.67), ('矩阵', 0.56), ('区域', 0.34), ('格子', 0.34), ('黑色', 0.3), ('异常情况', 0.3), ('画图', 0.26), ('绘制', 0.19)]
4294
[('身份', 1.18), ('加密狗', 1.02), ('资源', 0.73), ('Quota', 0.67), ('Sessi

[('语句', 1.19), ('粘贴', 0.89), ('Ctrl', 0.59), ('PLC', 0.54), ('黏贴', 0.34), ('任务栏', 0.3), ('复制粘贴', 0.27), ('框选', 0.24), ('快捷键', 0.23), ('批量', 0.17)]
4362
[('商品', 2.56), ('样式', 0.96), ('网站', 0.9), ('灰黑色', 0.6), ('色调', 0.54), ('加密狗', 0.39), ('CypNest', 0.23)]
4363
[('英文翻译', 1.03), ('英文', 0.7), ('中英对照', 0.52), ('语言包', 0.4), ('对话框', 0.4), ('附件', 0.39), ('中文', 0.37), ('意见', 0.37), ('语言', 0.37), ('名称', 0.26)]
4364
[('矩形', 0.85), ('mark', 0.84), ('特征', 0.74), ('线路板', 0.55), ('bcvmaster', 0.52), ('类型', 0.51), ('竞争力', 0.4), ('材料', 0.28)]
4365
[('规划', 1.11), ('缓冲区', 1.01), ('CAM', 0.66), ('序列', 0.5), ('原理', 0.42), ('Tubepro', 0.36), ('加速度', 0.33), ('方法', 0.29), ('轨迹', 0.26), ('距离', 0.24)]
4366
[('刀路', 2.16), ('Okay', 0.7), ('引线', 0.66), ('错误', 0.36), ('体验', 0.32)]
4367
[('指令', 1.06), ('浙江', 1.05), ('韦敏', 0.95), ('概率', 0.69), ('现象', 0.6), ('CypCut', 0.38)]
4368
[('Cypcut', 0.68), ('task', 0.61), ('html', 0.48), ('pms', 0.4), ('index', 0.4), ('php', 0.4), ('view', 0.4), ('鼠标', 0.36), ('引线', 0.3), ('

[('共享内存', 0.82), ('CPU', 0.49), ('进程', 0.45), ('权限', 0.37), ('实时性', 0.36), ('占用率', 0.35), ('hellp', 0.22), ('传递信息', 0.22), ('ring0', 0.22), ('ring3', 0.22)]
4440
[('角度', 3.62)]
4441
[('视窗', 1.55), ('高能', 0.44), ('幅面', 0.42), ('吴俊', 0.35), ('视屏', 0.35), ('机装', 0.33), ('朋友圈', 0.33), ('工人', 0.26), ('竖屏', 0.25), ('阶段', 0.25)]
4442
[('板材', 1.67), ('利用率', 0.9), ('一键', 0.8)]
4443
[('管子', 0.98), ('装夹', 0.77), ('中心', 0.55), ('高度', 0.48), ('字型', 0.47), ('视频', 0.3), ('平面', 0.27), ('卡盘', 0.22), ('图形', 0.15)]
4444
[('穿孔', 2.56), ('和松哥', 0.78), ('配置文件', 0.52)]
4445
[('结论', 0.4), ('程满', 0.23), ('HypCut', 0.2), ('终端用户', 0.2), ('决策', 0.2), ('经理', 0.19), ('复制粘贴', 0.19), ('小心', 0.18), ('批量生产', 0.18), ('意见', 0.17)]
4446
[('硬线', 1.33), ('石斌', 1.18), ('价格', 1.05), ('高器', 0.68), ('效率', 0.61)]
4447
[('余料', 1.28), ('现象', 0.44), ('卷尺', 0.39), ('张卫杰', 0.32), ('戴南', 0.26), ('人工', 0.26), ('图形', 0.23), ('圆弧', 0.23), ('绘制', 0.2), ('cypcut', 0.19)]
4448
[('边框', 0.99), ('包络线', 0.47), ('余料', 0.42), ('现象', 0.29), ('纯属',

[('BLT640', 2.39), ('穿孔', 1.6), ('BLT831', 0.78)]
4517
[('管心', 1.18), ('偏差', 0.74), ('角度', 0.66), ('管材', 0.59), ('矩形管', 0.44), ('节约', 0.44), ('效率', 0.28)]
4518
[('电源', 0.97), ('光闸', 0.92), ('错误', 0.66), ('cypcut', 0.6), ('鼠标', 0.48), ('恩耐', 0.43), ('刘希涛', 0.35), ('Tubepro', 0.28), ('开机', 0.26)]
4519
[('编辑器', 1.11), ('PLC', 1.03), ('微调', 0.44), ('同事', 0.42), ('基础', 0.37), ('开发人员', 0.34), ('市场部', 0.32), ('编辑', 0.27)]
4520
[('省料', 0.92), ('CypNest', 0.73), ('同参数', 0.65), ('板材', 0.46), ('案例', 0.43), ('套料', 0.42), ('间距', 0.3), ('尺寸', 0.29)]
4521
[('功率', 1.05), ('钢板', 0.53), ('选项', 0.51), ('拐角处', 0.5), ('拐角', 0.42), ('来源', 0.41), ('厚板', 0.39), ('平台', 0.25)]
4522
[('斯特', 3.12), ('时会', 1.21), ('场景', 0.84)]
4523
[('斯特', 1.04), ('和梅', 0.65), ('塞尔', 0.61), ('基本功能', 0.57), ('重量', 0.55), ('信息化', 0.5), ('传统', 0.47), ('数控系统', 0.47), ('HypCut', 0.27), ('附件', 0.25)]
4524
[('引线', 1.81), ('误报', 1.55), ('阳切会', 0.78), ('选项', 0.56)]
4525
[('提示框', 1.11), ('引线', 0.91), ('Toast', 0.65), ('制图', 0.57), ('弹框', 0.

[('气路', 0.97), ('气体', 0.84), ('床身', 0.59), ('延时', 0.46), ('高功率', 0.38), ('严巍', 0.26), ('床子', 0.24), ('氧气', 0.23), ('氮气', 0.23), ('空气', 0.22)]
4593
[('场景', 0.48), ('角钢', 0.47), ('边缘', 0.43), ('长条', 0.35), ('刀路', 0.34), ('绘制', 0.33), ('直线', 0.32), ('tubest', 0.27), ('矩形', 0.23), ('壁厚', 0.22)]
4594
[('memory', 0.57), ('电脑配置', 0.54), ('排序', 0.51), ('out', 0.47), ('卡顿', 0.39), ('报告单', 0.37), ('孩纸', 0.29), ('版面', 0.25), ('附件', 0.22), ('按钮', 0.22)]
4595
[('管道', 0.88), ('延时', 0.69), ('气体', 0.5), ('HypCut', 0.4), ('开气', 0.38), ('大功率', 0.38), ('气路', 0.36), ('氮气', 0.34), ('氧气', 0.34), ('类型', 0.23)]
4596
[('力矩', 1.82), ('卡时', 0.82), ('Who', 0.52), ('What', 0.52), ('Why', 0.51), ('测试人员', 0.44), ('地方', 0.39)]
4597
[('支架', 1.03), ('标定', 0.53), ('管子', 0.33), ('矩形管', 0.3), ('bug17102', 0.23), ('场景', 0.2), ('管材', 0.2), ('基本上', 0.18), ('基础', 0.13), ('测试人员', 0.12)]
4598
[('内膜', 1.16), ('正确处理', 0.7), ('阴阳', 0.7), ('方便管理', 0.67), ('零件库', 0.53), ('我会', 0.48), ('批量', 0.36), ('终端用户', 0.3), ('附件', 0.27), ('图纸',

In [70]:
sql = "select `id`, name, `desc` from zt_task where id > {0} limit 1000".format(0)
task_new = pd.read_sql(sql, engine)
from nohtml import strip_tags
task_new['desc'] = task_new['desc'].apply(lambda x:strip_tags(x))
task_new = task_new.loc[task_new['id'].drop_duplicates().index,:]

task_new = task_new.set_index(task_new['id'].values)

In [71]:
task_new['id']

1          1
2          2
3          3
4          4
5          5
        ... 
1001    1001
1002    1002
1003    1003
1004    1004
1005    1005
Name: id, Length: 1000, dtype: int64

In [72]:
task_new['id'] = task_new['id'].astype(int)

In [73]:
task_new

,id,name,desc
1,1,加密狗选型,选择一个合适的加密狗，带有硬件时钟，稳定性好，供应商信誉好，供货稳定，价格偏移。
2,2,循环拷机测试,使用测试用例中的循环拷机测试对Rev502.9版本的程序进行拷机测试，确认至少48小时内不会...
3,3,包络线计算,求单个图形或多个图形的包络线\r\n1.对单个图形，在给定精度条件下对原图采样，剔除原曲线中...
4,4,CypCut性能优化——打开文件优化,尽可能的提高CypCut打开文件的速度，包括文件读取、自动优化、曲线平滑、排序直到完全显示的过程。
5,5,澎凯加密沟通会议,和澎凯加密就加密狗的需求进行当面沟通。
...,...,...,...
1001,1001,123123,41241243214214\r\n2017年5月16日 19:01:483\r\n132
1002,1002,"轮岗测试-开启空移优化,蛙跳功能不撞头测试",由于不同的管子在空移优化的时会有撞头现象，\r\n测试点：管子包括矩形管，方形管，椭圆形管等...
1003,1003,完善智能空移的安全性,在HSG客户现场，切50x10的管，发生了2次撞头的现象。都和智能空移有关。希望彻查和理顺智...
1004,1004,完善双击备份文件恢复出厂参数的功能,HSG客户提出，非常希望有双击备份文件，恢复出厂参数的功能。就像cypcut一样。可以恢复机...


In [44]:
bug_new.loc[2207,:]

KeyError: 2207

In [277]:
es.index(index="pms_story_test", id=upload_df['story'][2], body= dict(upload_df.loc[2,['story']]))

RequestError: RequestError(400, 'illegal_argument_exception', 'field [spec] not present as part of path [spec]')

In [268]:
dict(story_df.loc[5,:])

{'story': 5,
 'title': 'CypOne动态速度算法',
 'spec': '1. 在不同速度下算法的切换。\r\n\xa0 \xa0 1）速度\r\n < 10mmm/s \r\n使用算法\r\n1。\r\n\r\n\r\n\xa0 \xa0 \xa02）\r\n速度\r\n >= 10mmm/s and \r\n速度\r\n < 20mmm/s\xa0 \r\n使用算法\r\n3\r\n但是关闭\r\n滤波器。\r\n\r\n\xa0 \xa0 3）速度\r\n >= 20mmm/s\r\n使用算法\r\n3 \r\n开启\r\n\r\n\r\n滤波器。\r\n\r\n2. 运动参数默认值。\r\n\xa0 \xa0\xa0\r\n\r\n3. 算法3的参数\r\n\xa0 \xa0 FBFW3.MaxAcc := \r\nWorkSpeed.Acc; \xa0 //\xa0最大加速度，默认为1200\r\n\xa0 \xa0 \xa0FBFW3.FIRFreq := 4; \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0//\xa0低通滤波频率\r\n\xa0 \xa0 \xa0FBFW3.ArcTolerance := 0.01; \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 // 圆弧控制精度\r\n\xa0 \xa0 \xa0FBFW3.CornerTolerance := 0.1; \xa0 \xa0 \xa0 \xa0 \xa0 //\xa0拐角控制精度\r\n\r\n\xa0 \xa0 \xa0\r\nFBFW3.SplitTime :=\r\n0.001; \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0//\xa0插补周期\r\n\r\n\r\n4. 运动参数使用\r\n\r\n\r\n\xa0 \xa0 \xa0点动，空移，走边框使用 \xa0空移\xa0加速度和加加速度。\r\n\r\n\r\n\xa0 \xa0 \xa0加工，前进/后退\r\n\r\n 使用\xa0\r\n加工\xa0加速度和加加速度。\r\n\xa0 \xa0 \xa0

In [199]:
dict(story_df.loc[1,:])

{'story': 1,
 'title': '加密狗选型',
 'spec': '选择一个合适的加密狗，带有硬件时钟，稳定性好，供应商信誉好，供货稳定，价格偏移。',
 'seg': {'加密狗': 0.2222222222222222,
  '选型': 0.1111111111111111,
  '硬件': 0.1111111111111111,
  '时钟': 0.1111111111111111,
  '稳定性': 0.1111111111111111,
  '信誉': 0.1111111111111111,
  '供货': 0.1111111111111111,
  '价格': 0.1111111111111111}}

In [160]:
upload_df

,story,title,spec,keywords
1,1,加密狗选型,选择一个合适的加密狗，带有硬件时钟，稳定性好，供应商信誉好，供货稳定，价格偏移。,"{'时钟': 6.02, '信誉': 6.02, '供货': 6.02, '加密狗': 5...."
2,2,特定情况下的外框最后排序问题,"类似于\n打标图层\n:\n开放一个\n最后切割图层\n, \n以弥补和应对\n新排序原则下...","{'原则': 6.94, '外框': 5.73, '排序': 3.52, '标图': 3.1..."
3,3,带膜切割开发一个直接跟随选项,目前带膜切割采用的是类似于定高切割规则执行，能满足大多数需求。\n但是如果切割板...,"{'水平度': 5.97, '曲面': 5.97, '带膜': 5.97, '规则': 3...."
4,4,双交换工作台的到位检测,\r\n需求由天琪、迪能提出\r\n\r\n目前双交换工作台是用手工方式来控制的，也就是客户...,"{'工作台': 2.96, '气缸': 2.53, '由天琪': 2.4, '手工': 2...."
5,5,CypOne动态速度算法,1. 在不同速度下算法的切换。\r\n 1）速度\r\n < 10mmm/s \r\n...,"{'FBFW3': 8.6, 'mmm': 6.9, '加速度': 4.14, '滤波器':..."
...,...,...,...,...
4637,4637,增加plc：所有B轴强制绝对空移,客户使用的密封销卡盘，需要卡盘移到到固定位置进行夹紧松开，于是配的PLC卡盘。\n但目前的B...,"{'密封': 10.8, '卡盘': 6.55, 'PLC': 1.91}"
4638,4638,支持读取驱动器的报警码,从空软陈宇轩的出差报告中得知：\n作为济南济南新天科技有限公司，我使用设备健康云时，监控到的...,"{'济南': 10.39, '陈宇轩': 6.66, '有限公司': 5.5, '科技': ..."
4639,4639,[预研需求]尖角入刀跟随不停顿,我是HypCut产品经理，我发现领创终端宝哥那边切三角形的零件时，在尖角入刀的时候，切割头判...,"{'尖角': 8.08, '原地': 5.92, '宝哥': 3.6, '三角形': 3.1..."
4640,4640,优化喷码参数设置界面。,【内部迭代需求】\n 基于功能点文档优化界面显示。功能点文档参考：http...,"{'喷码': 12.9, 'DPI': 4.26, 'BMH': 3.56, '标准': 1..."


In [73]:
dict(Counter({'a':0.5, 'b': 0.6 , 'c': 0.3}).most_common(2))

{'b': 0.6, 'a': 0.5}

In [95]:
upload_df

,story,title,spec
1,1,加密狗选型,"{'加密狗': 1.1278642909454892, '时钟': 0.8160080945..."
2,2,特定情况下的外框最后排序问题,"{'原则': 1.1755471563559279, '外框': 1.07032853717..."
3,3,带膜切割开发一个直接跟随选项,"{'规则': 0.8730295390670912, '板材': 0.83391739773..."
4,4,双交换工作台的到位检测,"{'工作台': 0.7695130578213172, '气缸': 0.5511045045..."
5,5,CypOne动态速度算法,"{'FBFW3': 1.1070768512401756, '加速度': 0.9101729..."
...,...,...,...
4637,4637,增加plc：所有B轴强制绝对空移,"{'卡盘': 1.9841320213145486, '密封': 1.46881457011..."
4638,4638,支持读取驱动器的报警码,"{'济南': 1.5184993837349057, '陈宇轩': 0.8610597731..."
4639,4639,[预研需求]尖角入刀跟随不停顿,"{'尖角': 1.4661989760199294, '原地': 0.88679008933..."
4640,4640,优化喷码参数设置界面。,"{'喷码': 2.248876446329006, 'DPI': 0.57600571377..."


In [34]:
import json
f = open("data\wordcount.json", 'w')
json.dump(words,f)
f.close()

with open("data\wordcount.json") as f:
    word_count = json.load(f)
word_count

{'n': 999,
 '加密狗': 12,
 '选型': 1,
 '硬件': 17,
 '时钟': 2,
 '稳定性': 13,
 '信誉': 2,
 '供货': 1,
 '价格': 3,
 '外框': 6,
 '排序': 35,
 '标图': 4,
 '图层': 31,
 '原则': 2,
 '图形': 149,
 '选项': 61,
 '规则': 12,
 '板材': 63,
 '水平度': 1,
 '曲面': 1,
 '模式': 49,
 '带膜': 1,
 '工作台': 29,
 '由天琪': 1,
 '迪能': 19,
 '手工': 4,
 '按钮': 65,
 '动作': 62,
 '钢板': 5,
 '气缸': 11,
 '条件': 15,
 '信号': 59,
 '行程开关': 3,
 '控制卡': 6,
 'CypOne': 5,
 '动态': 1,
 'mmm': 1,
 'and': 1,
 '滤波器': 1,
 'FBFW3': 1,
 'MaxAcc': 1,
 'WorkSpeed': 1,
 'Acc': 1,
 '加速度': 8,
 'FIRFreq': 1,
 '通滤波': 5,
 '频率': 14,
 'ArcTolerance': 1,
 '圆弧': 13,
 '控制精度': 3,
 'CornerTolerance': 1,
 '拐角': 13,
 'SplitTime': 1,
 '边框': 10,
 '程序': 42,
 '任务栏': 1,
 'Windows': 2,
 '对话框': 3,
 'CypCut': 71,
 'Cypcut': 29,
 '板子': 11,
 '角度': 33,
 '板边': 2,
 '距离': 37,
 '雷达': 2,
 '功率': 23,
 '曲线': 15,
 '方法': 16,
 '单调': 1,
 '半速': 1,
 '成功率': 2,
 '规律': 1,
 '方管': 51,
 '高度': 31,
 '圆管': 21,
 '灯具': 1,
 '艺术': 2,
 '造型': 1,
 '文字': 9,
 '平面': 30,
 '开机': 13,
 '画面': 8,
 '制造商': 42,
 '出厂': 8,
 '示教盒': 1,
 '认知度': 1,
 '加密': 15,
 'B

In [ ]:
len(word_count)

In [55]:
story_df =  pd.read_json("pms_story.json",encoding="utf-8", orient='records')
story_df

,title,spec
0,加密狗选型,"{'加密狗': 2, '选型': 1, '硬件': 1, '时钟': 1, '稳定性': 1..."
1,特定情况下的外框最后排序问题,"{'外框': 2, '排序': 3, '标图': 1, '图层': 1, '原则': 2, ..."
2,带膜切割开发一个直接跟随选项,"{'选项': 1, '规则': 2, '板材': 3, '水平度': 1, '曲面': 1,..."
3,双交换工作台的到位检测,"{'工作台': 5, '由天琪': 1, '迪能': 1, '手工': 2, '按钮': 3..."
4,CypOne动态速度算法,"{'CypOne': 1, '动态': 1, 'mmm': 4, 'and': 1, '滤波..."
...,...,...
6133,希望可以读取图纸中的零件编号，并在报告单中显示,"{'图纸': 2, '编号': 5, '报告单': 4, 'CypNest': 1, '文件..."
6134,图形修正界面增加文件名称的显示,"{'图形': 4, '修正': 4, '文件名称': 1, '体验': 1, '经理': 1..."
6135,希望可以支持生成单线的余料线,"{'单线': 2, '余料': 2, 'CypNest': 1, '红框': 1, '板材'..."
6136,希望手动绘制余料线时，可以增加撤销上一步功能,"{'手动': 3, '绘制': 5, '余料': 7, 'CypNest': 2}"


In [56]:
story_df['spec'][0]

{'加密狗': 2, '选型': 1, '硬件': 1, '时钟': 1, '稳定性': 1, '信誉': 1, '供货': 1, '价格': 1}

In [79]:

words
story_p = pd.DataFrame()


sql = 'select story, title, spec from zt_storyspec where story > {0} limit {1}'.format(4500, 1000)
story_new = pd.read_sql(sql, engine)
ifremain = (len(story_new) == 1000)
print(len(story_new))


from nohtml import strip_tags
story_new['spec'] = story_new['spec'].apply(lambda x:strip_tags(x))
story_new = story_new.loc[story_new['story'].drop_duplicates().index,:]

story_new = story_new.set_index(story_new['story'].values)





story_new['seg'] = ''
for i in story_new['story'].values:

    seg = segmentation((story_new['title'][i] + story_new['spec'][i]).split())
    word_count = Counter(seg)
    print(i)
    #print(word_count)
    wordn = sum(word_count.values())
    for word in word_count:
        word_count[word] = word_count[word] / wordn
    #print(word_count)

    words += Counter(word_count.keys())
    story_new['seg'][i] = dict(word_count)

story_p = pd.concat([story_p, story_new], axis = 0)



startid = story_new['story'].values[-1]

print(startid)








story_p.to_json('data/pms_story.json')

200
4501
4502
4503
4504
4505
4506
4507
4508
4509
4510
4511
4512
4513
4514
4515
4516
4517
4518
4519
4520
4521

d:\Users\fscut\anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



4522
4523
4524
4525
4526
4527
4528
4529
4530
4531
4532
4533
4534
4535
4536
4537
4538
4539
4540
4541
4542
4543
4544
4545
4546
4547
4548
4549
4550
4551
4552
4553
4554
4555
4556
4557
4558
4559
4560
4561
4562
4563
4564
4565
4566
4567
4568
4569
4570
4571
4572
4573
4574
4575
4576
4577
4578
4579
4580
4581
4582
4583
4584
4585
4586
4587
4588
4589
4590
4591
4592
4593
4594
4595
4596
4597
4598
4599
4600
4601
4602
4603
4604
4605
4606
4607
4608
4609
4610
4611
4612
4613
4614
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4630
4631
4632
4633
4634
4635
4636
4637
4638
4639
4640
4641
4642
4643
4644
4645
4646
4647
4648
4649
4650
4651
4652
4653
4653


In [80]:
story_p

,story,title,spec,seg
4501,4501,移植CypCut平台配置“激光器”功能,移植CypCut 762版本平台配置的激光器配置，激光器功率填写限制到2000W\n\n,"{'CypCut': 0.4, '平台': 0.4, '功率': 0.2}"
4502,4502,移植CypCut“冷却点”功能,为了满足1500A系统加工厚板的需求，移植移植CypCut“冷却点”功能，冷却点的添加、删除...,"{'CypCut': 0.4, '厚板': 0.2, '延时': 0.2, 'cypcut'..."
4503,4503,在CypOne软件的基础上进一步开放空移速度、空移加速度、低通,1500A软件在在CypOne软件的基础上进一步开放XY轴速度不限制、空移加速度最大1.5G...,"{'CypOne': 0.25, '基础': 0.25, '加速度': 0.375, 'BC..."
4504,4504,移植CypCut“一键保存故障信息”功能,http://pms.fscut.com/index.php?m=story&f=view&...,"{'CypCut': 0.058823529411764705, '一键': 0.11764..."
4505,4505,移植CypCut“任务”功能,为了提高客户加工中厚板的能力，移植CypCut“任务”功能。\n,"{'CypCut': 0.5, '中厚板': 0.25, '能力': 0.25}"
...,...,...,...,...
4649,4649,增加断点续喷的功能。,【内部迭代需求】\n 在喷码的过程中可能会遇到客户人为暂停或者停止，也可...,"{'断点': 0.15384615384615385, '续喷': 0.0769230769..."
4650,4650,提供只喷码，只加工，喷码并加工三种加工方式。,【内部迭代需求】\n 在切割系统中喷码是一个非必须的操作，即使文件里面包含...,"{'喷码': 0.4782608695652174, '文件': 0.13043478260..."
4651,4651,每个分组都能够设置定位补偿值,作为青虹的马工，我在设置4个分组进行定位切割时，发现各个分组切割后的误差情况并不相同。此时，...,"{'定位': 0.2727272727272727, '青虹': 0.09090909090..."
4652,4652,希望添加标准零件，可以实现两种不同的斜切,添加标准零件是比较常用的很好的功能。但是斜切仅限于“切断面沿z轴旋转”，不能沿X轴旋转，\n...,"{'标准': 0.2, '场景': 0.06666666666666667, '矩形管': ..."


In [82]:
story_p['story'].value_counts()

4607    1
4502    1
4509    1
4508    1
4507    1
       ..
4555    1
4554    1
4553    1
4552    1
4608    1
Name: story, Length: 153, dtype: int64